<a href="https://colab.research.google.com/github/igorlnunes/questoes_fisica/blob/main/Gerador_de_QuestoesDeFisica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Configura a API Key do Google Gemini
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# importa as bibliotecas do adk
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [5]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Markdown
import google.generativeai as genai
import json
import re
import asyncio
import textwrap

# Gera o enunciado, alternativas e a resposta correta para uma questão de física.
def gerar_enunciado_alternativas_resposta(topico: str, dificuldade: str) -> dict:
    model = genai.GenerativeModel("gemini-2.0-flash")
    prompt = f"""
    Gere uma questão de física de nível {dificuldade} sobre o tópico de {topico} para estudantes do ensino médio.
    A questão deve ser no formato de múltipla escolha com exatamente 4 opções (A, B, C, D).
    Indique claramente a resposta correta ao final da geração.

    Formato de saída desejado:
    {{
    "enunciado": "...",
    "opcoes": ["A) ...", "B) ...", "C) ...", "D) ..."],
    "resposta_correta": "..."
    }}
    """
    try:
        response = model.generate_content(prompt)
        json_string = response.text.strip()
        json_string = re.sub(r'^\s*```json\s*', '', json_string, flags=re.MULTILINE)
        json_string = re.sub(r'^\s*```\s*', '', json_string, flags=re.MULTILINE)
        questao_info = json.loads(json_string)
        return questao_info
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON da resposta: {e}")
        print(f"Resposta bruta do Gemini: {response.text}")
        return None
    except Exception as e:
        print(f"Erro ao gerar o enunciado, alternativas e resposta: {e}")
        return None

# Gera a resolução passo a passo para uma questão de física.
def gerar_resolucao_estruturada(enunciado: str, resposta_correta: str) -> str:
    model = genai.GenerativeModel("gemini-2.0-flash")
    prompt = f"""
    Gere uma resolução detalhada e passo a passo para a seguinte questão de física do ensino médio brasileiro.
    Organize a resolução em seções claras com títulos como: "1. Compreensão do Problema", "2. Identificação de Dados", "3. Fórmula Utilizada", "4. Passo a Passo da Solução", "5. Resposta Final".
    Seja conciso em cada seção, mas inclua todos os passos essenciais do raciocínio e utilize notação LaTeX para fórmulas e cálculos.

    {enunciado}

    A resposta correta para esta questão é a opção {resposta_correta}.
    """
    try:
        response = model.generate_content(prompt)
        resolucao = response.text
        resolucao = resolucao.replace(r'\(', r'$').replace(r'\)', r'$').replace(r'\[', r'$$').replace(r'\]', r'$$')
        return resolucao
    except Exception as e:
        print(f"Erro ao gerar a resolução: {e}")
        return None

dificuldade_dropdown = widgets.Dropdown(options=['Iniciante', 'Intermediário', 'Avançado'], description='Dificuldade:')
topico_dropdown = widgets.Dropdown(options=['Cinemática', 'Ondulatória', 'Dinâmica'], description='Tópico:')
gerar_questao_button = widgets.Button(description="Gerar Questão")
resposta_button = widgets.Button(description="Revelar Resposta")
resolucao_button = widgets.Button(description="Mostrar Resolução")
output = widgets.Output()

# Exibe string Markdown e renderiza MathJax.
def display_markdown_with_mathjax(markdown_string: str):
    display(HTML(markdown_string))

# Lida com o clique do botão de gerar questão.
def on_gerar_questao_button_click(b):
    clear_output(wait=True)
    with output:
        display(HTML("Gerando questão..."))
        questao = gerar_enunciado_alternativas_resposta(topico_dropdown.value, dificuldade_dropdown.value)
        if questao:
            global questao_data
            questao_data = questao
            display_markdown_with_mathjax(f"Enunciado:<br>{questao['enunciado']}")
            for opcao in questao['opcoes']:
                display_markdown_with_mathjax(f"{opcao}")
            display(resposta_button)
        else:
            display(HTML("Erro ao gerar a questão."))

# Lida com o clique do botão de revelar resposta.
def on_resposta_button_click(b):
    clear_output(wait=True)
    with output:
        display_markdown_with_mathjax(f"Resposta Correta:<br>{questao_data['resposta_correta']}")
        display(resolucao_button)

# Lida com o clique do botão de mostrar resolução.
def on_resolucao_button_click(b):
    clear_output(wait=True)
    with output:
        display(HTML("Gerando resolução..."))
        resolucao = gerar_resolucao_estruturada(questao_data['enunciado'], questao_data['resposta_correta'])
        if resolucao:
            display(to_markdown(f"Resolução:\n{resolucao}"))
        else:
            display(HTML("Erro ao gerar a resolução."))

gerar_questao_button.on_click(on_gerar_questao_button_click)
resposta_button.on_click(on_resposta_button_click)
resolucao_button.on_click(on_resolucao_button_click)

display(dificuldade_dropdown)
display(topico_dropdown)
display(gerar_questao_button)
display(output)